In [40]:
import tensorflow as tf
import csv
import numpy as np
from PIL import Image

from keras import Model
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from keras.utils import Sequence
from keras.backend import epsilon
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.io

import pandas as pd
import numpy as np

np.random.seed(seed=1)

In [54]:
mat['box_coord'][0][0]

42

In [10]:
mat.items()

dict_items([('__header__', b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Jun 12 05:08:31 2019'), ('__version__', '1.0'), ('__globals__', []), ('box_coord', array([[ 42,  99,  15, 149]], dtype=uint8)), ('obj_contour', array([[  2.96536524,   7.55604534,  19.45780856,  33.73992443,
         49.552267  ,  81.68702771,  97.1593199 , 113.14168766,
        127.93387909, 134.05478589, 129.80415617, 114.16183879,
        109.06108312, 101.75      , 108.72103275,  99.02959698,
         89.33816121,  94.60894207,  86.27770781,  71.48551637,
         65.19458438,  57.54345088,  65.53463476,  71.14546599,
         70.97544081,  60.94395466,  50.23236776,  49.552267  ,
         31.52959698,  31.69962217,  28.80919395,  26.5988665 ,
         25.91876574,  35.27015113,  34.42002519,  22.0081864 ,
          8.74622166,   3.64546599,   1.26511335,   2.62531486,
          2.62531486],
       [ 23.63413098,  32.13539043,  33.83564232,   8.84193955,
          2.04093199,   6.63161209,   1.3608312

In [41]:
df_labels = pd.read_csv('D:/Ganga/AIML/AIML_ML/Hackathon/train_labels.csv')

In [42]:
df_labels.head()

,ImageName,Annotation,Class
0,./images/3917.jpg,./annotations/3917.mat,58
1,./images/5368.jpg,./annotations/5368.mat,85
2,./images/1327.jpg,./annotations/1327.mat,18
3,./images/5544.jpg,./annotations/5544.mat,88
4,./images/6145.jpg,./annotations/6145.mat,95


In [43]:
df_labels.shape

(4346, 3)

In [44]:
print('Total count of Images: ', df_labels.shape[0])
print('Number of Image Category: ', len(df_labels.Class.unique()))

Total count of Images:  4346
Number of Image Category:  97


In [61]:
ImgCnt = df_labels.shape[0]
coords = np.zeros((ImgCnt, 4))

for i in range(ImgCnt):
    try:
        mat = scipy.io.loadmat(df_labels.Annotation[i])
        coords[i, 0] = mat['box_coord'][0][0]
        coords[i, 1] = mat['box_coord'][0][1]
        coords[i, 2] = mat['box_coord'][0][2]
        coords[i, 3] = mat['box_coord'][0][3]
    except Exception as e:
        print(e)

In [71]:
from keras.applications.mobilenet import MobileNet, preprocess_input
from PIL import Image


X_train = np.zeros((ImgCnt, 128, 128, 3), dtype=np.float32)

for i in range(ImgCnt):
    try:
        img = Image.open(df_labels.ImageName[i])
        img = img.resize((128, 128)) # Resize image
        img = img.convert('RGB')
        X_train[i] = (preprocess_input(np.array(img, dtype=np.float32)))
    except Exception as e:
        print(e)

In [66]:
model = MobileNet(input_shape=(128, 128, 3), include_top=False, alpha=1.0) # Load pre-trained mobilenet
# Do not include classification (top) layer

# to freeze layers, except the new top layer, of course, which will be added below
for layer in model.layers:
    layer.trainable = False

# Add new top layer which is a conv layer of the same size as the previous layer so that only 4 coords of BBox can be output
x = model.layers[-1].output
x = Conv2D(4, kernel_size=4, name="coords")(x)
# In the line above kernel size should be 3 for img size 96, 4 for img size 128, 5 for img size 160 etc.
x = Reshape((4,))(x) # These are the 4 predicted coordinates of one BBox

model = Model(inputs=model.input, outputs=x)

In [67]:
def loss(gt,pred):
    intersections = 0
    unions = 0
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height
    
    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection

#     Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
        if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
            intersections += intersection[j]
            unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + epsilon()), 4)
    iou = iou.astype(np.float32)
    return iou

def IoU(y_true, y_pred):
    iou = tf.py_func(loss, [y_true, y_pred], tf.float32)
    return iou

In [73]:
gt = coords
model.compile(optimizer='Adam', loss='mse', metrics=[IoU]) # Regression loss is MSE

#checkpoint = ModelCheckpoint("model-{val_iou:.2f}.h5", verbose=1, save_best_only=True,
#                              save_weights_only=True, mode="max", period=1) # Checkpoint best validation model
#stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max") # Stop early, if the validation error deteriorates
#reduce_lr = ReduceLROnPlateau(monitor="val_iou", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")
# Reduce learning rate if Validation IOU does not improve

model.fit(X_train,gt,
            epochs=5,batch_size = 32,
            verbose=1)

Epoch 1/5
4346/4346 [==============================] - 140s 32ms/step - loss: 1308.6345 - IoU: 0.2853
Epoch 2/5
4346/4346 [==============================] - 140s 32ms/step - loss: 907.7382 - IoU: 0.3390
Epoch 3/5
4346/4346 [==============================] - 140s 32ms/step - loss: 757.5893 - IoU: 0.3588
Epoch 4/5
4346/4346 [==============================] - 138s 32ms/step - loss: 669.6443 - IoU: 0.3835
Epoch 5/5
4346/4346 [==============================] - 139s 32ms/step - loss: 608.0318 - IoU: 0.4029


In [75]:
df_Testlabels = pd.read_excel('D:/Ganga/AIML/AIML_ML/Hackathon/test_labels_students.xlsx')

In [79]:
ImgCntTest = df_Testlabels.shape[0]

X_test = np.zeros((ImgCntTest, 128, 128, 3), dtype=np.float32)

for i in range(ImgCntTest):
    try:
        img = Image.open(df_Testlabels.ImageName[i])
        img = img.resize((128, 128)) # Resize image
        img = img.convert('RGB')
        X_test[i] = (preprocess_input(np.array(img, dtype=np.float32)))
    except Exception as e:
        print(e)

In [78]:
len(X_test)

1863

In [81]:
predictions_test = model.predict(X_test)

In [87]:
predictions_test[0].tolist()

[26.43086814880371, 211.79676818847656, 48.07197570800781, 227.60491943359375]

In [86]:
predictions_test.tolist()[0]

[26.43086814880371, 211.79676818847656, 48.07197570800781, 227.60491943359375]

In [91]:
df_Testlabels.to_csv('./reslab1.csv')

In [92]:
df_test.to_csv('./reslab2.csv')

In [88]:
df_test = pd.DataFrame(predictions_test)

In [89]:
df_test.head()

,0,1,2,3
0,26.430868,211.796768,48.071976,227.604919
1,9.716758,228.122101,41.056602,300.111938
2,29.100285,190.792984,69.997917,198.027649
3,64.747185,246.127930,33.784031,262.783875
4,12.975296,96.984955,20.237215,272.839508


In [32]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_std, labels, test_size=0.2, random_state=2)
print ("No. of images in train dataset: ", len(X_train_split))
print ("No. of images in Validation dataset: ", len(X_val_split))

No. of images in train dataset:  3476
No. of images in Validation dataset:  870


In [21]:
model = Sequential()
model.add(Conv2D(filters=32, input_shape = (128,128,3), kernel_size=5)) 
model.add(Conv2D(filters=64, kernel_size=3))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(Dense(units = 97, activation = 'softmax'))

In [22]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train_split, y_train_split,
                    epochs=10, 
                    validation_data=(X_val_split, y_val_split),
                    verbose = 1,
                    batch_size=128)

In [25]:
from keras.applications import InceptionResNetV2

conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))

219062272/219055592 [==============================] - 420s 2us/step


In [33]:
model = Sequential()
model.add(conv_base)
#model.add(Conv2D(filters=32, input_shape = (128,128,3), kernel_size=5)) 
#model.add(Conv2D(filters=64, kernel_size=3))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(Dense(units = 97, activation = 'softmax'))

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 3, 3, 1536)        54336736  
_________________________________________________________________
flatten_4 (Flatten)          (None, 13824)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               3539200   
_________________________________________________________________
dense_8 (Dense)              (None, 97)                24929     
Total params: 57,900,865
Trainable params: 57,840,321
Non-trainable params: 60,544
_________________________________________________________________


In [35]:
print ("No. of trainable weights before freezing the conv_base: ", len(model.trainable_weights))
conv_base.trainable = False
print ("No. of trainable weights after freezing the conv_base: ", len(model.trainable_weights))

No. of trainable weights before freezing the conv_base:  492
No. of trainable weights after freezing the conv_base:  4


In [36]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [37]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
train_datagen.fit(X_train_split)

In [38]:
val_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
val_datagen.fit(X_val_split)

In [39]:
train_generator = train_datagen.flow(X_train_split, y_train_split, batch_size=9)
val_generator = val_datagen.flow(X_val_split, y_val_split, batch_size=9)

In [ ]:
model.fit_generator(train_generator,\
                    epochs=10, steps_per_epoch=3476//9, \
                    verbose=1,validation_data=val_generator, validation_steps = 870//9)

Epoch 1/10
283/386 [====================>.........] - ETA: 5:28 - loss: 4.6076 - acc: 0.0616